In [7]:
!yes | pip uninstall torchvison
!pip install -qU torchvision

yes: standard output: Broken pipe


In [2]:
!pip install wandb -qqq

# MNIST Training using PyTorch

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()

## Data
### Getting the data



In [4]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [5]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-447341124968/sagemaker/DEMO-pytorch-mnist


## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [12]:
# !pygmentize mnist.py
!pygmentize src/mnist_wandb.py

import argparse
import json
import logging
import os
import sys
import wandb # add for wandb

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 

In [13]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

wandb: Currently logged in as: tiptopbin. Use `wandb login --relogin` to force relogin


In [ ]:
# Or set your key manually
# current_api_key=xxx

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [14]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist_wandb.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=2,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    },
                   # Pass Weights & Biases API Key as an environment variable
                   # You can find the key by going to https://wandb.ai/authorize
                   environment={"WANDB_API_KEY": current_api_key})

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [15]:
estimator.fit({'training': inputs})

2022-06-23 02:53:37 Starting - Starting the training job...
2022-06-23 02:54:02 Starting - Preparing the instances for trainingProfilerReport-1655952817: InProgress
......
2022-06-23 02:55:03 Downloading - Downloading input data...
2022-06-23 02:55:23 Training - Downloading the training image...
2022-06-23 02:56:02 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-23 02:56:01,163 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-23 02:56:01,165 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-23 02:56:01,173 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-23 02:56:01,179 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-23 02:56:01,467 sagemaker-training-toolkit INFO     Install

Experiment with five learning rates and see their effects on performance.

In [16]:
lrs = [0.01, 0.001, 0.05, 0.1, 0.2]
for lr in lrs:
    estimator = PyTorch(entry_point='mnist_wandb.py',
                        source_dir="src",
                        role=role,
                        py_version='py3',
                        framework_version='1.8.0',
                        instance_count=2,
                        instance_type='ml.c5.2xlarge',
                        hyperparameters={
                            'epochs': 10,
                            'backend': 'gloo',
                            'lr': lr,
                        },
                       environment={"WANDB_API_KEY": current_api_key})
    estimator.fit({'training': inputs}, wait=False)

In [17]:
estimator.fit({'training': inputs})

2022-06-23 03:01:55 Starting - Starting the training job...ProfilerReport-1655953314: InProgress
......
2022-06-23 03:03:00 Starting - Preparing the instances for training......
2022-06-23 03:04:25 Downloading - Downloading input data......
2022-06-23 03:05:20 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-23 03:05:18,471 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-23 03:05:18,473 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-23 03:05:18,481 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-23 03:05:18,488 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-23 0

## Host
### Create endpoint
After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single ```ml.m4.xlarge``` instance.

In [10]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-------!

### Evaluate

You can use the test images to evalute the endpoint. The accuracy of the model depends on how many it is trained. 

In [14]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [15]:
import gzip 
import numpy as np
import random
import os

data_dir = 'data/MNIST/raw'
with gzip.open(os.path.join(data_dir, "t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1, 28, 28).astype(np.float32)

mask = random.sample(range(len(images)), 16) # randomly select some of the test images
mask = np.array(mask, dtype=np.int)
data = images[mask]

In [18]:
response = predictor.predict(np.expand_dims(data, axis=1))
print("Raw prediction result:")
print(response)
print()

labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions: ")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

Raw prediction result:
[[ -841.81384277  -809.42578125  -686.82019043  -514.44500732
   -308.11486816  -555.79718018  -855.24853516   -68.2052002
   -401.67419434     0.        ]
 [ -646.59735107  -895.97802734     0.          -544.23052979
   -755.7565918   -922.04547119  -625.824646    -812.57006836
   -500.33776855  -805.3817749 ]
 [ -887.1842041   -817.05535889  -741.71582031  -870.87322998
      0.          -643.92150879  -453.70858765  -701.31427002
   -686.24975586  -333.54837036]
 [ -891.87609863  -857.50964355  -627.76599121  -970.46557617
      0.          -768.94299316  -505.81027222  -653.79400635
   -676.14233398  -344.46173096]
 [ -660.29199219  -792.37634277  -664.95281982     0.
   -958.40881348  -385.23153687  -958.44628906  -779.3081665
   -500.71435547  -699.39404297]
 [ -834.11621094  -886.70550537  -529.53649902  -998.31347656
   -532.21209717  -538.67773438     0.         -1198.16943359
   -689.94897461  -867.32391357]
 [ -465.91723633     0.          -236.6927795

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [24]:
sagemaker_session.delete_endpoint(
    endpoint_name = predictor.endpoint_name
)